In [1]:
### Loading data from S3
import urllib

ACCESS_KEY = 'SECRET_KEY = 'ENCODED_SECRET_KEY = urllib.quote(SECRET_KEY, '')
AWS_BUCKET_NAME = 'train-raw.microsoft'
MOUNT_NAME = 'train-raw'

dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
# Load all the files from S3 into files RDD

# Persist files RDDs 

In [2]:
# This takes funckin a lot of time. 
file_infos = dbutils.fs.ls("/mnt/train-raw")


In [3]:
### Refine the file_list to get only the malware id
def map_file_id(x):
  return str(x[1])

file_list = map(map_file_id, file_infos)


In [4]:
# Map this into pair RDDs: key/value -> file_name/content
file_ids = sc.parallelize(file_list)

#test_ids = sc.parallelize(['01azqd4InC7m9JpocGv5.asm', '01azqd4InC7m9JpocGv5.bytes', '0A32eTdBKayjCWhZqDOQ.asm', '0A32eTdBKayjCWhZqDOQ.bytes'])

# Anonymous functions
def get_number_lines(file):
    
    file_type = ''
    num_lines = count_new_line_character(file[1])
    
    if (re.search('.asm', file[0]) is not None):
        file_type = '.asm: '
    elif(re.search('.bytes', file[0]) is not None):
        file_type = '.bytes'
    
    return (get_fileID_from_file_path(file[0]), {file_type: num_lines})
  
def get_content(file_name):
    return (file_name, get_file_content_as_string(file_name))


pair_RDD = file_ids.map(get_content)
pair_RDD = pair_RDD.map(get_number_lines)

result = pair_RDD.collect()

# pair_RDD.persist() # Persist data to local storage to speed up access


In [5]:
list_result = ''.join(re.sub('[(|)]', '', str(e)) + '\n' for e in result)

upload_content_as_string_to_s3(list_result, 'train_processed.csv')

In [6]:
# Functions that support some common tasks
import re 
import boto
from boto.s3.connection import OrdinaryCallingFormat
from boto.s3.key import Key

# Utility functions
def get_fileID_from_file_path(path):
    '''Get input as path of .asm or .bytes files and
    
       Return the id

       E.g: get_fileID_from_file_path('home/livetolove128/01azqd4InC7m9JpocGv5.asm'): '01azqd4InC7m9JpocGv5'
    '''
    temp_str = re.search('.{20}\.(asm|bytes)', path).group(0)
    str_id = re.search('.{20}', temp_str).group(0)
    return str_id

def count_new_line_character(string):
    '''Get input as a string and
       
       Return the number of new line characters
       
       E.g: count_new_line_character('nguyen \n heelllo \n'): 2
    '''
    return len(re.findall('\\n', string))
  
def get_file_content_as_string(file_name):
  # Establish connection with s3
  s3 = boto.s3.connect_to_region(
  region_name = 'us-west-1',
  aws_access_key_id = ACCESS_KEY,
  aws_secret_access_key = SECRET_KEY,
  calling_format = OrdinaryCallingFormat())
  train_raw = s3.get_bucket('train-raw.microsoft', validate=False)
  content = Key(train_raw, file_name).get_contents_as_string()
  return content

def upload_content_as_string_to_s3(content_str, filename):
   # Establish connection with s3
  s3 = boto.s3.connect_to_region(
  region_name = 'us-west-1',
  aws_access_key_id = ACCESS_KEY,
  aws_secret_access_key = SECRET_KEY,
  calling_format = OrdinaryCallingFormat())
  train_processed = s3.get_bucket('train-processed.microsoft', validate=False)
  k = Key(train_processed)
  k.key = filename
  k.set_contents_from_string(content_str)
  return content_str



In [7]:
# Real data processing will happen here
